In [ ]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

X_train = torch.tensor([
    [-1.2,3.1],
    [-0.9,2.9],
    [-0.5,2.6],
    [2.3,-1.1],
    [2.7,-1.5]
])
y_train = torch.tensor([0,0,0,1,1])

X_test = torch.tensor([
    [-0.8,2.8],
    [2.6,-1.6],
])
y_test = torch.tensor([0,1])

class ToyDataset(Dataset):
    def __init__(self,X,y):
        self.features = X
        self.labels = y
    
    def __getitem__(self, index): # 用于检索单个数据记录及其对应标签的指令
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x,one_y
    
    def __len__(self):
        return self.labels.shape[0] # 用于返回数据集总长度的指令

train_ds = ToyDataset(X_train,y_train)
test_ds = ToyDataset(X_test,y_test)

# print(len(train_ds))
# print(len(test_ds))

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_ds, # 之前创建的ToyDataset实例作为数据加载器的输入
    batch_size=2, #  关键参数：批量大小为2,每次迭代返回两个样本
    shuffle=True, # 是否打乱数据
    num_workers=0, # 后台进程的数量
    drop_last=True # 丢弃每个训练轮次的最后一个批次
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False, # 没有必要打乱测试数据
    num_workers=0
)

# # 假设我们有一个数据集，DataLoader设置了batch_size=2，并且shuffle=True。
# # 假设数据集有6个样本，特征和标签都是标量（为了简单），比如：
# # 样本0: (0.1, 0)
# # 样本1: (0.2, 1)
# # 样本2: (0.3, 0)
# # 样本3: (0.4, 1)
# # 样本4: (0.5, 0)
# # 样本5: (0.6, 1)
# # 由于shuffle=True，每个epoch开始时数据顺序会被打乱。
# # 假设第一次epoch的顺序被随机打乱为[样本1, 样本4, 样本0, 样本5, 样本2, 样本3]。
# # 那么，DataLoader会按照batch_size=2来分组：
# # 第一批: [样本1, 样本4] -> 特征: [0.2, 0.5], 标签: [1, 0]
# # 第二批: [样本0, 样本5] -> 特征: [0.1, 0.6], 标签: [0, 1]
# # 第三批: [样本2, 样本3] -> 特征: [0.3, 0.4], 标签: [0, 1]
# # 所以，循环会输出：
# # Batch0: 特征tensor([0.2, 0.5]), 标签tensor([1, 0])
# # Batch1: 特征tensor([0.1, 0.6]), 标签tensor([0, 1])
# # Batch2: 特征tensor([0.3, 0.4]), 标签tensor([0, 1])
for idx,(x,y) in enumerate(train_loader): # 遍历数据加载器
    print(f"Batch{idx}",x,y)

In [ ]:
import torch
import torch.nn.functional as F
from torch.autograd import grad
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class NeuralNetwork(torch.nn.Module):
    def __init__(self,num_inputs,num_outputs): 
        super().__init__()
        self.layers = torch.nn.Sequential(
            #1st hidden layer
            torch.nn.Linear(num_inputs,30), 
            torch.nn.ReLU(), 

            #2nd hidden layer
            torch.nn.Linear(30,20), 
            torch.nn.ReLU(),

            #output layer
            torch.nn.Linear(20,num_outputs),
        )

    def forward(self,x):
        logits = self.layers(x) 
        return logits

torch.manual_seed(123)

model = NeuralNetwork(num_inputs=2,num_outputs=2) #数据集包含 2 个特征和 2 个类别

optimizer = torch.optim.SGD(model.parameters(),lr=0.5) 
#让优化器知道需要优化哪些参数,学习率（lr）设置为 0.5

num_epochs = 3

X_train = torch.tensor([
    [-1.2,3.1],
    [-0.9,2.9],
    [-0.5,2.6],
    [2.3,-1.1],
    [2.7,-1.5]
])
y_train = torch.tensor([0,0,0,1,1])

X_test = torch.tensor([
    [-0.8,2.8],
    [2.6,-1.6],
])
y_test = torch.tensor([0,1])

class ToyDataset(Dataset):
    def __init__(self,X,y):
        self.features = X
        self.labels = y
    
    def __getitem__(self, index): # 用于检索单个数据记录及其对应标签的指令
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x,one_y
    
    def __len__(self):
        return self.labels.shape[0] # 用于返回数据集总长度的指令

train_ds = ToyDataset(X_train,y_train)
test_ds = ToyDataset(X_test,y_test)

train_loader = DataLoader(
    dataset=train_ds, # 之前创建的ToyDataset实例作为数据加载器的输入
    batch_size=2, #  关键参数：批量大小为2,每次迭代返回两个样本
    shuffle=True, # 是否打乱数据
    num_workers=0, # 后台进程的数量
    drop_last=True # 丢弃每个训练轮次的最后一个批次
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False, # 没有必要打乱测试数据
    num_workers=0
)

for epoch in range(num_epochs):
    model.train()
    for batch_idx,(features,labels) in enumerate(train_loader):
        
        logits = model(features)
        loss = F.cross_entropy(logits,labels)

        optimizer.zero_grad() # 将上一轮的梯度设置为零，以防止意外的梯度累积
        loss.backward() #计算损失函数相对于模型参数的梯度
        optimizer.step() #优化器使用梯度来更新模型参数
        #  对于这里用到的 SGD 优化器，它的调整方法很简单：
        # 它会把给出的梯度乘以一个“学习率”，这个“学习率”决定了每次调整的幅度有多大。
        # 然后，它会朝着让错误减少的方向（梯度的反方向）稍微调整一下模型的参数。

        print(f"Epoch : {epoch+1:03d}/{num_epochs:03d}"
              f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
              f" | Train Loss : {loss:.2f}")
    
    model.eval()

with torch.no_grad():
    outputs = model(X_train)
print(outputs)

torch.set_printoptions(sci_mode=False) #使输出更易读
probas = torch.softmax(outputs,dim=1)
print(probas)

predictions1 = torch.argmax(probas,dim=1)
print(predictions1)

predictions2 = torch.argmax(outputs,dim=1)
print(predictions2)

predictions1 == y_train

torch.sum(predictions1 == y_train)

In [ ]:
import torch
import torch.nn.functional as F
from torch.autograd import grad
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class NeuralNetwork(torch.nn.Module):
    def __init__(self,num_inputs,num_outputs): 
        super().__init__()
        self.layers = torch.nn.Sequential(
            #1st hidden layer
            torch.nn.Linear(num_inputs,30), 
            torch.nn.ReLU(), 

            #2nd hidden layer
            torch.nn.Linear(30,20), 
            torch.nn.ReLU(),

            #output layer
            torch.nn.Linear(20,num_outputs),
        )

    def forward(self,x):
        logits = self.layers(x) 
        return logits

torch.manual_seed(123)

model = NeuralNetwork(num_inputs=2,num_outputs=2) #数据集包含 2 个特征和 2 个类别

optimizer = torch.optim.SGD(model.parameters(),lr=0.5) 
#让优化器知道需要优化哪些参数,学习率（lr）设置为 0.5

num_epochs = 3

X_train = torch.tensor([
    [-1.2,3.1],
    [-0.9,2.9],
    [-0.5,2.6],
    [2.3,-1.1],
    [2.7,-1.5]
])
y_train = torch.tensor([0,0,0,1,1])

X_test = torch.tensor([
    [-0.8,2.8],
    [2.6,-1.6],
])
y_test = torch.tensor([0,1])

class ToyDataset(Dataset):
    def __init__(self,X,y):
        self.features = X
        self.labels = y
    
    def __getitem__(self, index): # 用于检索单个数据记录及其对应标签的指令
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x,one_y
    
    def __len__(self):
        return self.labels.shape[0] # 用于返回数据集总长度的指令

train_ds = ToyDataset(X_train,y_train)
test_ds = ToyDataset(X_test,y_test)

train_loader = DataLoader(
    dataset=train_ds, # 之前创建的ToyDataset实例作为数据加载器的输入
    batch_size=2, #  关键参数：批量大小为2,每次迭代返回两个样本
    shuffle=True, # 是否打乱数据
    num_workers=0, # 后台进程的数量
    drop_last=True # 丢弃每个训练轮次的最后一个批次
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False, # 没有必要打乱测试数据
    num_workers=0
)

for epoch in range(num_epochs):
    model.train()
    for batch_idx,(features,labels) in enumerate(train_loader):
        
        logits = model(features)
        loss = F.cross_entropy(logits,labels)

        optimizer.zero_grad() # 将上一轮的梯度设置为零，以防止意外的梯度累积
        loss.backward() #计算损失函数相对于模型参数的梯度
        optimizer.step() #优化器使用梯度来更新模型参数
        #  对于这里用到的 SGD 优化器，它的调整方法很简单：
        # 它会把给出的梯度乘以一个“学习率”，这个“学习率”决定了每次调整的幅度有多大。
        # 然后，它会朝着让错误减少的方向（梯度的反方向）稍微调整一下模型的参数。

        print(f"Epoch : {epoch+1:03d}/{num_epochs:03d}"
              f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
              f" | Train Loss : {loss:.2f}")
    
    model.eval()

def compute_accuracy(model,dataloader):

    model = model.eval()
    correct = 0.0
    total_example = 0

    for idx,(features,labels) in enumerate(dataloader):
        with torch.no_grad():
            logits = model(features)

        predictions = torch.argmax(logits,dim=1)
        compare = labels == predictions #返回一个由 True/False 值组成的张量，取决于标签是否匹配
        correct += torch.sum(compare) #sum 操作会计算 True 值的数量
        total_example += len(compare)
    
    return (correct / total_example).item() 
    # 这是正确预测的比例，一个介于 0 和 1 之间的值。
    # 并且 .item() 返回张量的值作为 Python 浮点数。

print(compute_accuracy(model,train_loader))
print(compute_accuracy(model,test_loader))

In [1]:
import torch
print(torch.backends.mps.is_available())
tensor_1 = torch.tensor([1., 2., 3.])
tensor_2 = torch.tensor([4., 5., 6.])
tensor_1 = tensor_1.to("mps")
tensor_2 = tensor_2.to("mps")
print(tensor_1 + tensor_2)

True
tensor([5., 7., 9.], device='mps:0')


In [1]:
import torch
import torch.nn.functional as F
from torch.autograd import grad
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class NeuralNetwork(torch.nn.Module):
    def __init__(self,num_inputs,num_outputs): 
        super().__init__()
        self.layers = torch.nn.Sequential(
            #1st hidden layer
            torch.nn.Linear(num_inputs,30), 
            torch.nn.ReLU(), 

            #2nd hidden layer
            torch.nn.Linear(30,20), 
            torch.nn.ReLU(),

            #output layer
            torch.nn.Linear(20,num_outputs),
        )

    def forward(self,x):
        logits = self.layers(x) 
        return logits

torch.manual_seed(123)

model = NeuralNetwork(num_inputs=2,num_outputs=2) #数据集包含 2 个特征和 2 个类别

# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device = torch.device("mps")
model = model.to(device)

optimizer = torch.optim.SGD(model.parameters(),lr=0.5) 
#让优化器知道需要优化哪些参数,学习率（lr）设置为 0.5

num_epochs = 3

X_train = torch.tensor([
    [-1.2,3.1],
    [-0.9,2.9],
    [-0.5,2.6],
    [2.3,-1.1],
    [2.7,-1.5]
])
y_train = torch.tensor([0,0,0,1,1])

X_test = torch.tensor([
    [-0.8,2.8],
    [2.6,-1.6],
])
y_test = torch.tensor([0,1])

class ToyDataset(Dataset):
    def __init__(self,X,y):
        self.features = X
        self.labels = y
    
    def __getitem__(self, index): # 用于检索单个数据记录及其对应标签的指令
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x,one_y
    
    def __len__(self):
        return self.labels.shape[0] # 用于返回数据集总长度的指令

train_ds = ToyDataset(X_train,y_train)
test_ds = ToyDataset(X_test,y_test)

train_loader = DataLoader(
    dataset=train_ds, # 之前创建的ToyDataset实例作为数据加载器的输入
    batch_size=2, #  关键参数：批量大小为2,每次迭代返回两个样本
    shuffle=True, # 是否打乱数据
    num_workers=0, # 后台进程的数量
    drop_last=True # 丢弃每个训练轮次的最后一个批次
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False, # 没有必要打乱测试数据
    num_workers=0
)

for epoch in range(num_epochs):
    model.train()
    for batch_idx,(features,labels) in enumerate(train_loader):
        
        features,labels = features.to(device), labels.to(device)
        logits = model(features)
        loss = F.cross_entropy(logits,labels)

        optimizer.zero_grad() # 将上一轮的梯度设置为零，以防止意外的梯度累积
        loss.backward() #计算损失函数相对于模型参数的梯度
        optimizer.step() #优化器使用梯度来更新模型参数
        #  对于这里用到的 SGD 优化器，它的调整方法很简单：
        # 它会把给出的梯度乘以一个“学习率”，这个“学习率”决定了每次调整的幅度有多大。
        # 然后，它会朝着让错误减少的方向（梯度的反方向）稍微调整一下模型的参数。

        print(f"Epoch : {epoch+1:03d}/{num_epochs:03d}"
              f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
              f" | Train Loss : {loss:.2f}")
    
    model.eval()

def compute_accuracy(model,dataloader):

    model = model.eval()
    correct = 0.0
    total_example = 0

    for idx,(features,labels) in enumerate(dataloader):
        with torch.no_grad():
            logits = model(features)

        predictions = torch.argmax(logits,dim=1)
        compare = labels == predictions #返回一个由 True/False 值组成的张量，取决于标签是否匹配
        correct += torch.sum(compare) #sum 操作会计算 True 值的数量
        total_example += len(compare)
    
    return (correct / total_example).item() 
    # 这是正确预测的比例，一个介于 0 和 1 之间的值。
    # 并且 .item() 返回张量的值作为 Python 浮点数。

print(compute_accuracy(model,train_loader))
print(compute_accuracy(model,test_loader))

: 